# Convolution Neural Network을 감성분석에 적용하기
### Dr. Seong-K. Park@ai-khwarizmi.com

### 요약 정리

<p>
인터넷과 소셜 미디어 시대에 사람들의 견해와 리뷰, 추천은 정치나 비지니스에 상당히 중요한 자원이 되고있습니다. 현대 기술에 감사하게도 이제 우리는 이런 데이터를 대단히 효율적으로 수집하고 분석할 수 있는 기술이 생겼습니다. 이번 장에서는 감성분석이라고 불리는 자연어 처리(Natural Language Processing, NLP)의 상세 영역까지 들어가서 문서를 분류하기 위한 자연어 처리(NLP) 를 위한 컨볼류션알고리즘의 사용법을 설명합니다.</p>

<p>최근에 많은 애플리케이션에서 사용하는 데이터가 있는가 있는데, 바로 텍스트입니다. 스팸 메일 분류를 예로 들면, 이메일의 내용에 이 분류 작업에 필요한 중요한 정보가 들어 있습니다. 또는 이민 정책에 관한 정치인의 의견을 분석해야 할 때 각자의 언행이나 트윗이 중요한 정보를 제공합니다. 고객 서비스에서는 메시지가 불만사항인지 문의사항인지를 구분해야 할 때가 많습니다. 메시지의 제목이나 내용으로 고객의 의도를 자동으로 파악해서 적절한 부서로 전달하거나, 완전히 자동으로 응답할 수 도 있습니다.</p>

<p>
이번 장에서는 인터넷 영화 자료(Internet Movie Database, IMDb)의 영화 리뷰 데이터로 작업을 할 것이다. 이 데이터는 매스 등에(Mass et al.)의해 수집되었다. 영화 리뷰 데이터는 5만 개로 양이나 음으로 레이블된 양극의 영화 리뷰로 구성되어 있다(여기서, 별 6개이상은 양(+) 그리고 아니면 음(-)). 다음 절에서는 이러한 영화 리뷰의 부분집합에서 의미 있는 축출하여 특정 리뷰어가 영화에 대해 좋다고 하는지 싫다고 하는지를 예측할 수 있는 머신러닝 모델을 만드는 방법에 대해 학습한다.</p>


<p>
영화 리뷰 데이터의 압축된 아카이브(84.1MB) 다운로드 장소: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
<br>
데이터를 성공적으로 추출했다면 압축이 풀린 다운로드 아카이브에서 하나의 CSV 파일로 개별 텍스트 문서들을 병합하겠습니다.
</p>

In [14]:
import pandas as pd
import os

import numpy as np


Imdb_file = './Imdb_data.csv'

if os.path.exists(Imdb_file):
    
    df = pd.read_csv(Imdb_file)
    
    print(df.head(10))

else:
    labels = {'pos':1, 'neg':0}

    df = pd.DataFrame()

    for s in {'test', 'train'}:
        for l in ('pos', 'neg'):
            path = './aclImdb/%s/%s' % (s, l)
            for file in os.listdir(path):
                with open(os.path.join(path, file), 'r') as infile:
                    txt = infile.read()

                df = df.append([[txt, labels[l]]], ignore_index=True)

    df.columns = ['review','sentiment']

    np.random.seed(0)

    df = df.reindex(np.random.permutation(df.index))
    df.to_csv(Imdb_file, index=False)
    
    df = pd.read_csv(Imdb_file)
    print(df.head(3))


                                              review  sentiment
0  On MTV cribs all the ballers and shot callers ...          1
1  This film is really ONLY Bill Maher's interpre...          0
2  On the scale of 1 to 10, I gave this a 4. I th...          0
3  Okay okay, I must admit, I do somewhat like Pe...          1
4  This is the worst movie I've ever seen. Boring...          0
5  The movie is really cool, I thought. It sticks...          1
6  I guess this is the first movie that made me a...          1
7  <br /><br />This movie is by far one of my fav...          1
8  In the year 2000 (keep in mind, this is two ye...          1
9  Walking With Dinosaurs is an amazing Documenta...          1


In [13]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 2 * 32
embedding_dims = 50
filters = 250

kernel_size = 3
hidden_dims = 250
epochs = 2


In [5]:
print('Loading data...')

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences'); print(len(x_test), 'test sequences')

print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))

Loading data...
25000 train sequences
25000 test sequences
218
189
141


In [6]:
print('Pad sequences (samples x time)')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = sequence.pad_sequences( x_test, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))

Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
400
400
400


#### 영화 리뷰을 위한 CNN model 만들기

In [7]:
print('Build model...')
model = Sequential()


model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))

model.add(Conv1D(filters, kernel_size, padding='same', activation='relu'))
                 
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 400, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0     

#### Train 된 영화 리뷰 자료를 test 셋에 적용하기

In [10]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.99%


In [12]:
y_hat = model.predict_classes(x_test)

test_false = [im for im in zip(x_test,y_hat,y_test) if im[1] != im[2]]
test_true = [im for im in zip(x_test,y_hat,y_test) if im[1] == im[2]]

print('\ntest_false: ',len(test_false))
print('test_true: ',len(test_true))

24992/25000 [============================>.] - ETA: 0s
test_false:  2753
test_true:  22247
